# Traductores e interpretes dirigidos por sintaxis
Notas de clase sobre Teoría de la Compilación

**Juan David Velásquez Henao**   
jdvelasq@unal.edu.co  
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

[Licencia](https://github.com/jdvelasq/teoria-de-la-compilacion/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/teoria-de-la-compilacion/blob/master/readme.md)

**Software utilizado**.

> Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org), en el cual se presenta una introducción al diseño de lectores, generadores, traductores, interpretes y compiladores. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Windows, Linux y OS X.

> Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

> Haga clic [aquí](http://nbviewer.jupyter.org/github/jdvelasq/teoria-de-la-compilacion/blob/master/tcc-06-traductores.ipynb) para ver la última versión de este documento en nbviewer.

> Descargue la última versión de este documento, los archivos de datos y los programas en Python a su disco duro; luego, carguelos y ejecutelos en línea en [Try Jupyter!](https://try.jupyter.org)

#### Contenido

>* [Tipos de analizadores top-down](#Tipos-de-analizadores-top-down)
* [Transformación de la gramática](#Transformación-de-la-gramática)
* [Remoción de recursión por la izquierda.](#Remoción-de-recursión-por-la-izquierda.)
* [Factorización por la izquierda.](#Factorización-por-la-izquierda.)
* [Traducción de notación infija a postfija o polaca inversa](#Traducción-de-notación-infija-a-postfija-o-polaca-inversa)
   * [infix2postfix.py](#infix2postfix.py) 
   * [Utilidad dc de Unix](#Utilidad-dc-de-Unix)
* [Interprete dirigido por sintaxis I](#Interprete-dirigido-por-sintaxis-I)
* [Interprete dirigido por sintaxis II](#Interprete-dirigido-por-sintaxis-II)
* [Generación del árbol sintáctico](#Generación-del-árbol-sintáctico)
    * [infix2ast.py](#infix2ast.py)
    * [ast2prefix](#ast2prefix)
    * [ast2postfix](#ast2postfix)
    * [ast2infix](#ast2infix)

# Tipos de analizadores top-down

[Contenido](#Contenido)

Analizador recursivo descendente (Codificación a mano muy simple).

* Analizador LL(1): (**L**eft–to–right, **L**eft-recursive, **1** token–ahead).  

* Analizador LL(K): (**L**eft–to–right, **L**eft-recursive, **K** tokens–ahead).

**Ejercicio**  
Use la siguiente gramática para analizar la expresión `1 + 2 * 3`.

    
    exp:     term expp

    expp:    ‘+’ term expp  |  ε

    term:    factor termp

    termp:   ‘*’ factor termp |  ε

    factor:  NUM | ‘(’ exp ‘)’

# Transformación de la gramática

[Contenido](#Contenido)

Considere la siguiente gramática:

    exp:     exp addop term  |  term 
    
    addop:   ‘+’ 

    term:    term mulop factor | factor 

    mulop:   ‘*’
    
    factor:  ‘(’ exp ‘)’ | NUM

Ella puede ser reescrita como:


    exp → term [addop term]* → term [‘+’ term]*

    term → factor [mulop factor]* → factor [‘*’ factor]*

    factor → ‘(’ exp ‘)’ | NUM

# Remoción de recursión por la izquierda.

[Contenido](#Contenido)

Un conjunto de producciones de la forma:

$A \to A \alpha_1  \; | \; A \alpha_2 \; | \; \dots  \; | \; A \alpha_n \; | \; 
\beta_1 \; | \; \beta_2 \; | \; \dots \; |  \; \beta_m $

Puede reescribirse como:
 
 
$ A \to \beta_1 A' \; | \; \ldots \; | \; \beta_m A' $

$ A' \to \alpha_1 A'  \; | \; \alpha_2 A' \; | \; \ldots \; | \; \alpha_n A' \; | \; \epsilon $
 

**Ejemplo**    
La producción

    exp: exp '+' term  |  exp '–' term   |  term
    
puede reescribirse como:


	exp:   term expp
	expp:  '+' term expp  |  '–' term expp  | ε

# Factorización por la izquierda.

[Contenido](#Contenido)

Se requiere cuando dos o más reglas comparten un prefijo común.

La regla $A \to \alpha \; \beta \; | \; \alpha \; \gamma$ se reescribe como: 
 
$A \to \alpha \; A'$

$A' \to \beta \;|\; \gamma$

**Ejemplo:**  
La regla

	if-stmt: IF '(' exp ')' stmt
	       | IF '(' exp ')' stmt ELSE stmt

se reescribe como:

	if-stmt:    IF ‘(’ exp ‘)’ stmt  else-part
	else-part:  ELSE stmt | ε

# Traducción de notación infija a postfija o polaca inversa

[Contenido](#Contenido)

Analice la conversión de notación matemática infija a polaca inversa. 
¿Cómo se puede realizar la traducción?

```
1 + 2 + 3	      →  1 2 + 3 +

1 * 2 + 3 * 4      →  1 2 * 3 4 * +

(1 + 2) * (3 + 4)  →  1 2 + 3 4 + *

(para el último caso)
    MLT
      +-----ADD
      |     +---- NUM: 1
      |     |
      |     +---- NUM: 2
      |
      +---- ADD
            +---- NUM: 3
            |
            +---- NUM: 4

```

## infix2postfix.py

[Contenido](#Contenido)

In [1]:
import sys, shlex
import dataTree as dt

In [2]:
def infix2postfix(text):

    ## prepara las estructuras de datos
    DATA = dt.TreeNode('DATA')
    TOKENTABLE = dt.SubNode(DATA, 'TOKENTABLE')

    ##
    ## analizador léxico
    ##
    def yylex(text):
    
        ## parte *generica* para todos los analizadores lexicos
        def put_token(token, lexeme):
            #
            dt.SubNode(TOKENTABLE, '', dict(token = token, lexeme = lexeme))
            #

        TOKENTABLE.clear() # borra la tabla de tokens si existe

        
        ## ejecución del analisis lexico
        lexemes = list(shlex.shlex(text))
        for lexeme in lexemes:

            if lexeme in '+ - * / ( )'.split():
                put_token(lexeme, lexeme)
            elif lexeme.isdigit():
                put_token('NUM', lexeme)
            else:
                print('Caracter invalido:', lexeme)
                sys.exit()

        put_token('EOI', 'EOI')

        
    ##
    ## funciones genericas requeridas por
    ## el parser durante el reconocimiento de
    ## la gramatica
    ##
    def yytext(offset=0):
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('lexeme')
        #
    
    def yytoken():
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('token')
        #
    
    def yymatch(token, offset=0):
        #
        return token == yytoken()
        #

    def yyadvance(offset=1):
        #
        TOKENTABLE.attrib['index'] += offset
        #

    def yyaccept(token, advance=True):
        #
        if not yymatch(token):
            print('Syntax error')
            sys.exit()
            
        if advance:
            yyadvance()

    ##
    ## Analizador sintáctico
    ##
    def yyparse():
        """
        gramatica:

            exp:     term expp
            expp:    '+' term expp | '-' term expp | `eps`
            term:    factor termp
            termp:   '*' factor termp | '/' factor termp | `eps`
            factor:  NUM  |  '('  exp  ')'

        """
        def expr():
            term()
            expp()

        def expp():
            if yymatch('+') or yymatch('-'):
                op = yytext()
                yyadvance()
                term()
                print(op, ' ', sep= '', end='')
                expp()

        def term():
            factor()
            termp()

        def termp():
            if yymatch('*') or yymatch('/'):
                op = yytext()
                yyadvance()
                factor()
                print(op, ' ', sep= '', end='')
                termp()

        def factor():
            if yymatch('NUM'):
                print(yytext(), ' ', sep='', end='')
                yyadvance()
            elif yymatch('('):
                yyadvance()
                expr()
                yyaccept(')')
            else:
                print('Syntax error')
                sys.exit()

        # 'index' almacena el indice del token
        # actual en la tabla de tokens
        TOKENTABLE.set('index', 0)
        expr()
        print("")
        del TOKENTABLE.attrib['index']


    yylex(text)
    yyparse()

In [3]:
infix2postfix('(1 + 2) * (3 + 4) * (5 + 6)')

1 2 + 3 4 + * 5 6 + * 


## Utilidad dc de Unix

[Contenido](#Contenido)

Un evaluador de expresiones en notación polaca inversa en fácilmente programable – utilidad dc de Unix.  `bc` es una utilidad que recibe notación polaca, la transforma a poláca reversa y usa a dc como calculadora.

Gramática de `dc`.

```
expr  : cmd expr

cmd   : NUM    # introduce el número NUM en el stack
      | ‘+’      
      | ‘-’
      | ‘*’
      | ‘/’ 
      | ‘p’    # imprime la cima del stack
      | ‘q’    # quit

```

# Interprete dirigido por sintaxis I

[Contenido](#Contenido)

Este interprete retorna resultados a través de la gramática.


Expresión y árbol sintáctico:
    
    (1 + 2) * (3 + 4)

    MLT, value = 21
      +---------ADD, value = 3
      |          +-------- NUM: 1, value = 1
      |          |
      |          +---------NUM: 2, value = 2
      |
      +-------- ADD, value = 7
                 +-------- NUM: 3, value = 3
                 |
                 +-------- NUM: 4, value = 4

In [4]:

def infixcalc(text):
    '''
    infixcal:
       interprete que devuelve valores a través del parser
    '''
    
    ## estructuras de datos
    DATA = dt.TreeNode('DATA')
    TOKENTABLE = dt.SubNode(DATA, 'TOKENTABLE')

    ##
    ## Analizador lexico
    ##
    def yylex(text):
        ##
        ## parte *generica* para todos los analizadores lexicos
        ##
        def put_token(token, lexeme):
            dt.SubNode(TOKENTABLE, '', dict(token = token, lexeme = lexeme))

        TOKENTABLE.clear() # borra la tabla de tokens si existe

        ##
        ## analisis lexico
        ##
        lexemes = list(shlex.shlex(text))
        for lexeme in lexemes:
            if lexeme in '+ - * / ( )'.split():
                put_token(lexeme, lexeme)
            elif lexeme.isdigit():
                put_token('NUM', lexeme)
            else:
                print('Caracter invalido:', lexeme)
                sys.exit()
        put_token('EOI', 'EOI')


    ## funciones genericas requeridas por
    ## el parser durante el reconocimiento de
    ## la gramatica
    def yytext(offset=0):
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('lexeme')
        #
    
    def yytoken():
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('token')
        #

    def yymatch(token, offset=0):
        #
        return token == yytoken()
        #

    def yyadvance(offset=1):
        #
        TOKENTABLE.attrib['index'] += offset
        #

    def yyaccept(token, advance=True):
        #
        if not yymatch(token):
            print('Syntax error')
            sys.exit()
        #
        if advance:
            yyadvance()

    ##
    ## Analizador sintáctico
    ##
    def yyparse():
        '''
        gramatica:
            exp:    term expp
            expp:   '+' term expp | '-' term expp | `eps`
            term:   factor termp
            termp:  '*' factor termp | '/' factor termp | `eps`
            factor: NUM  |  '('  exp  ')'
        '''
        
        ##
        ## exp:    term expp
        ##
        def expr():
            root = term()
            right = expp(root)
            if right is None:
                return root
            return right

        ##
        ## expp:   '+' term expp | '-' term expp | `eps`
        ##
        def expp(left):
            if yymatch('+') or yymatch('-'):
                op = yytext()
                yyadvance()
                right = term()
                if op == '+':
                    root = left + right
                else:
                    root = left - right
                m = expp(root)
                if m is None:
                    return root
                return m
            return None

        ##
        ## term:   factor termp
        ##
        def term():
            root = factor()
            right = termp(root)
            if right is None:
                return root
            return right

        ##
        ## termp:  '*' factor termp | '/' factor termp | `eps`
        ##
        def termp(left):
            if yymatch('*') or yymatch('/'):
                op = yytext()
                yyadvance()
                right = factor()
                if op == '*':
                    root = left * right
                else:
                    root = left / right
                m = termp(root)
                if m is None:
                    return root
                return m
            return None

        ##
        ## factor: NUM  |  '('  exp  ')'
        ##
        def factor():
            if yymatch('NUM'):
                value = int(yytext())
                yyadvance()
                return value
            if yymatch('('):
                yyadvance()
                value = expr()
                yyaccept(')')
                return value
            print('Syntax error')
            sys.exit()

        ## `index` almacena el indice del token
        ## actual en la tabla de tokens
        TOKENTABLE.set('index', 0)
        print(expr())
        del TOKENTABLE.attrib['index']

    ## rutina principal
    yylex(text)
    yyparse()


In [5]:
infixcalc('1 + 2 + 3 + 4')

10


In [6]:
infixcalc('1 * 2 * 3 * 4')

24


In [7]:
infixcalc('(1 + 2) * (3 + 4)')

21


# Interprete dirigido por sintaxis II

[Contenido](#Contenido)

Este interprete realiza los cálculos usando un stack de operandos.

    (1 + 2) * (3 + 4)

    MLT
      | {MLT = PUSH (POP * POP)}
      +---------ADD
      |          | {ADD = PUSH (POP + POP)} 
      |          +-------- NUM: 1 {PUSH 1}
      |          |
      |          +---------NUM: 2 {PUSH 2}
      |
      +-------- ADD
                 | {ADD = PUSH (POP + POP)}
                 +-------- NUM: 3 {PUSH 3}
                 |
                 +-------- NUM: 4 {PUSH 4}

In [8]:
def infixstack(text):
    '''
    infixstack
        interprete que realiza cálculos usando un stack
    '''
    
    ## prepara las estructuras de datos
    DATA = dt.TreeNode('DATA')
    TOKENTABLE = dt.SubNode(DATA, 'TOKENTABLE')
    
    ##
    ## Analizador lexico
    ##
    def yylex(text):
        ## parte *generica* para todos los analizadores lexicos
        def put_token(token, lexeme):
            dt.SubNode(TOKENTABLE, '', dict(token = token, lexeme = lexeme))

        TOKENTABLE.clear() # borra la tabla de tokens si existe

        ## analisis lexico
        lexemes = list(shlex.shlex(text))
        for lexeme in lexemes:
            if lexeme in '+ - * / ( )'.split():
                put_token(lexeme, lexeme)
            elif lexeme.isdigit():
                put_token('NUM', lexeme)
            else:
                print('Caracter invalido:', lexeme)
                sys.exit()
        put_token('EOI', 'EOI')

    ## funciones genericas requeridas por
    ## el parser durante el reconocimiento de
    ## la gramatica
    def yytext(offset=0):
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('lexeme')
        #

    def yytoken():
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('token')
        #

    def yymatch(token, offset=0):
        #
        return token == yytoken()
        #

    def yyadvance(offset=1):
        #
        TOKENTABLE.attrib['index'] += offset
        #

    def yyaccept(token, advance=True):
        #
        if not yymatch(token):
            print('Syntax error')
            sys.exit()
        #
        if advance:
            yyadvance()

    ##
    ## Analizador sintáctico
    ##
    def yyparse():
        """
        gramatica:
            exp:    term expp
            expp:   '+' term expp | '-' term expp | `eps`
            term:   factor termp
            termp:  '*' factor termp | '/' factor termp | `eps`
            factor: NUM  |  '('  exp  ')'
        """

        ## stack de operandos simulada usando una lista
        stack = []
        
        ##
        ## exp:    term expp
        ##
        def expr():
            term()
            expp()

        ##
        ## expp:   '+' term expp | '-' term expp | `eps`
        ##
        def expp():
            if yymatch('+') or yymatch('-'):
                
                ##
                op = yytext()
                yyadvance()
                term()
                
                ## los resultados están en la cima del stack
                arg2 = stack.pop()
                arg1 = stack.pop()
                if op == '+':
                    stack.append(arg1 + arg2)
                else:
                    stack.append(arg1 - arg2)
                
                ##
                expp()

        ##
        ## term:   factor termp
        ##
        def term():
            factor()
            termp()

        ##
        ## termp:  '*' factor termp | '/' factor termp | `eps`
        ##
        def termp():
            if yymatch('*') or yymatch('/'):
                
                ##
                op = yytext()
                yyadvance()
                factor()
                
                ## los resultados están en la cima del stack
                arg2 = stack.pop()
                arg1 = stack.pop()
                if op == '*':
                    stack.append(arg1 * arg2)
                else:
                    stack.append(arg1 / arg2)
                    
                ##
                termp()

        ##
        ## factor: NUM  |  '('  exp  ')'
        ##
        def factor():
            
            ##
            if yymatch('NUM'):
                stack.append(int(yytext()))
                yyadvance()
                return
            
            ##
            if yymatch('('):
                yyadvance()
                expr()
                yyaccept(')')
                return
            
            ##
            print('Syntax error')
            sys.exit()
    
        ## aquí comienza la componente principal 
        ## del analizador sintáctico
    
        ## `index` almacena el indice del token
        ## actual en la tabla de tokens
        TOKENTABLE.set('index', 0)
        expr()
        del TOKENTABLE.attrib['index']
        return stack[-1]


    ## programa principal
    yylex(text)
    result = yyparse()
    print(result)


In [9]:
infixstack('1 + 2 + 3 + 4')

10


In [10]:
infixstack('1 * 2 * 3 * 4')

24


In [11]:
infixstack('(1 + 2) * (3 + 4)')

21


# Generación del árbol sintáctico

[Contenido](#Contenido)

## infix2ast.py

[Contenido](#Contenido)

El siguiente código construye el arbol sintáctico en memoria.

La función `infix2ast` genera el árbol sintáctico en memoria y luego lo imprime.

In [12]:
def infix2ast(text):
    '''
    infix2ast
        crea un arbol sintactico a partir de la
        gramatica de una calculadora simple (+, -, *, /)
    '''

    ## prepara las estructuras de datos
    DATA = dt.TreeNode('DATA')
    TOKENTABLE = dt.SubNode(DATA, 'TOKENTABLE')
    SYNTAXTREE = dt.SubNode(DATA, 'SYNTAXTREE')

    ##
    ## analizador lexico
    ##
    def yylex(text):
        # parte *generica* para todos los analizadores lexicos
        def put_token(token, lexeme):
            dt.SubNode(TOKENTABLE, '', dict(token = token, lexeme = lexeme))

        TOKENTABLE.clear() # borra la tabla de tokens si existe

        # analisis lexico
        lexemes = list(shlex.shlex(text))
        for lexeme in lexemes:
            if lexeme in '+ - * / ( )'.split():
                put_token(lexeme, lexeme)
            elif lexeme.isdigit():
                put_token('NUM', lexeme)
            else:
                print('Caracter invalido:', lexeme)
                sys.exit()
        put_token('EOI', 'EOI')

    
    ## funciones genericas requeridas por
    ## el parser durante el reconocimiento de
    ## la gramatica
    def yytext(offset=0):
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('lexeme')
        #

    def yytoken():
        #
        return TOKENTABLE[TOKENTABLE.get('index')].get('token')
        #

    def yymatch(token, offset=0):
        #
        return token == yytoken()
        #

    def yyadvance(offset=1):
        #
        TOKENTABLE.attrib['index'] += offset
        #

    def yyaccept(token, advance=True):
        #
        if not yymatch(token):
            print('Syntax error')
            sys.exit()
        #
        if advance:
            yyadvance()

    ##
    ## analizador sintactico
    ##
    def yyparse():
        """
        gramatica:
            exp:    term expp
            expp:   '+' term expp | '-' term expp | `eps`
            term:   factor termp
            termp:  '*' factor termp | '/' factor termp | `eps`
            factor: NUM  |  '('  exp  ')'
        """
        ## 
        ## exp:    term expp
        ##
        def expr():
            root = term()
            right = expp(root)
            if right is None:
                return root
            return right

        ##
        ## expp:   '+' term expp | '-' term expp | `eps`
        ##
        def expp(left):
            if yymatch('+') or yymatch('-'):
                root = dt.TreeNode(yytext())
                yyadvance()
                right = term()
                root.append(left)
                root.append(right)
                m = expp(root)
                if m is None:
                    return root
                return m
            return None

        ##
        ## term:   factor termp
        ##
        def term():
            root = factor()
            right = termp(root)
            if right is None:
                return root
            return right

        ##
        ## termp:  '*' factor termp | '/' factor termp | `eps`
        ##
        def termp(left):
            if yymatch('*') or yymatch('/'):
                root = dt.TreeNode(yytext())
                yyadvance()
                right = factor()
                root.append(left)
                root.append(right)
                m = termp(root)
                if m is None:
                    return root
                return m
            return None

        ##
        ## factor: NUM  |  '('  exp  ')'
        ##
        def factor():

            if yymatch('NUM'):
                node = dt.TreeNode('NUM', dict(text=yytext(), value = int(yytext())))
                yyadvance()
                return node

            if yymatch('('):
                yyadvance()
                node = expr()
                yyaccept(')')
                return node

            print('Syntax error')
            sys.exit()

        # `index` almacena el indice del token
        # actual en la tabla de tokens
        TOKENTABLE.set('index', 0)
        SYNTAXTREE.append(expr())
        del TOKENTABLE.attrib['index']    
    
    
    yylex(text)
    yyparse()
    return SYNTAXTREE
   

In [13]:
st = infix2ast('(1+2) * (3 + 4) * (5 +6)')
dt.printTree(st)

+-- SYNTAXTREE
    +-- *
        +-- *
        |   +-- +
        |   |   +-- NUM {text: 1, value: 1}
        |   |   +-- NUM {text: 2, value: 2}
        |   +-- +
        |       +-- NUM {text: 3, value: 3}
        |       +-- NUM {text: 4, value: 4}
        +-- +
            +-- NUM {text: 5, value: 5}
            +-- NUM {text: 6, value: 6}


## ast2prefix

[Contenido](#Contenido)

Una vez construido el árbol sintáctico este puede ser transformado de muchas maneras. Por ejemplo, en el siguiente código se convierte la expresión aritmética a notación prefija a partir del árbol.

In [14]:
st = infix2ast('1 + 2 + 3')
dt.printTree(st)

+-- SYNTAXTREE
    +-- +
        +-- +
        |   +-- NUM {text: 1, value: 1}
        |   +-- NUM {text: 2, value: 2}
        +-- NUM {text: 3, value: 3}


Se imprime la expresión resultante a medida que se recorre el árbol de arriba a abajo así:

      nodo:                salida:
      [000] SYNTAXTREE
      [001] +              (+
      [002] +              (+ (+ 
      [003] NUM            (+ (+ 1
      [004] NUM            (+ (+ 1 2
      [002] +              (+ (+ 1 2)
      [005] NUM            (+ (+ 1 2) 3
      [001] +              (+ (+ 1 2) 3 )

In [15]:
def ast2prefix(root):
    ## 
    ## esta es la función recursiva que recorre los nodos del arbol
    ##
    def print_node(node):
        
        ## El nodo contiene un operador aritmético
        if node.tag in '+ * - /'.split():
            
            ## imprime el `(` en la entrada a la función
            print( '(', node.tag, ' ',  sep = '',  end = '')
            
            ## recorre los hijos
            for n in node._children:
                print_node(n)
                
            ## imprime el `)` después de recorrer los hijos
            print(') ', end = '')
            
        ## el nodo contiene un número
        elif node.tag == 'NUM':
            
            print(' ', node.attrib['value'], sep = '', end = '')
                  
        else:
            ## procesa los nodos hijos
            for n in node._children:
                print_node(n)
                  
    ## componente principal
    print_node(root)
    print('')

In [16]:
ast2prefix(st)

(+ (+  1 2)  3) 


## ast2postfix

[Contenido](#Contenido)

La expresión aritmética también puede escribirse en notación postfija mediante el recorrido del árbol. La diferencia está en la ubicación del código donde se imprime dentro de la función que recorre el árbol.

In [17]:
st = infix2ast('1 + 2 + 3')
dt.printTree(st)

+-- SYNTAXTREE
    +-- +
        +-- +
        |   +-- NUM {text: 1, value: 1}
        |   +-- NUM {text: 2, value: 2}
        +-- NUM {text: 3, value: 3}


Se imprime la expresión resultante a medida que se recorre el árbol de arriba a abajo así:

      nodo:                salida:
      [000] SYNTAXTREE
      [001] +              (
      [002] +              (( 
      [003] NUM            (( 1
      [004] NUM            (( 1 2
      [002] +              (( 1 2 +)
      [005] NUM            (( 1 2 +) 3
      [001] +              (( 1 2 +) 3 +)

In [18]:
def ast2postfix(root):
    
    ## esta es la función recursiva que recorre el árbol
    def print_node(node):
        
        ## el nodo contiene un operador aritmético.
        if node.tag in '+ * - /'.split():
            
            print( '(', sep = '',  end = '')
            for n in node._children:
                print_node(n)
            
            print('', node.tag, ') ', sep = '', end = '')
            
        ## el nodo contiene un número.
        elif node.tag == 'NUM':
            
            print(node.attrib['value'], ' ', sep = '', end = '')
        
        ## procesa los hijos
        else:
            
            for n in node._children:
                print_node(n)
                
    ## aca se inicia el recorrido del arbol
    print_node(root)
    print('')

In [19]:
ast2postfix(st)

((1 2 +) 3 +) 


## ast2infix

[Contenido](#Contenido)

Se procede de igual manera que en los ejemplos anteriores.

In [20]:
def ast2infix(root):
    
    ## funcion genérica que recorre el arbol
    def print_node(node):
        
        ## el nodo contiene un operador aritmético.
        if node.tag in '+ * - /'.split():
            
            ## imprime el `(`
            print('(', sep='', end='')

            ## procesa los hijos
            print_node(node._children[0])
            
            ## imprime el espacio en blanco de separación
            print(' ', node.tag, ' ',  sep='',  end='')
            print_node(node._children[1])
            
            ## imprime el `)`
            print(')', sep='', end='')
            
        ## el nodo contiene un número
        elif node.tag == 'NUM':
            
            print(node.attrib['value'], sep='', end='')
            
        else:
            
            for n in node._children:
                print_node(n)
                
    print_node(root)
    print('')

In [21]:
ast2infix(st)

((1 + 2) + 3)


---

[Contenido](#Contenido)